In [8]:
import load
from plot import plot3D

# from mygp import mygp

import numpy as np
import pandas as pd
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process import kernels
from sklearn.preprocessing import StandardScaler

import skopt 
from skopt.space import Real

from icecream import ic


In [9]:
class mygp(object):

    def __init__(
        self,  
        gp=None,  
        **kwargs    
    ):
    
        if gp:
            self.gp = gp    
        else:
            self.gp = GaussianProcessRegressor()    



        self.ndims=1
        self.sample_X = np.array([])
        self.sample_y = np.array([])

        
        self.kwargs = kwargs
        self.kappa = kwargs.pop("kappa",0.0)
        self.acquisition_function = kwargs.pop("acquisition_function","LCB")    
        self.xi = kwargs.pop("xi", 0.01)     


    def prn(self, **kwargs):


        print(f"ndims = {self.ndims}. sample shape = {self.sample_X.shape}")        
        print(f"kappa={self.kappa}")
        print(f"xi={self.xi}")
        print(f"acquisition_function={self.acquisition_function}")    
        print(self.sample_X)
        print(self.sample_y)   

        return

    def sample_init(self,X,y):

        X_shape = X.shape
        X_len =  len(X_shape)

        
        if X_len == 1:
            self.ndims = 1
        elif X_len == 2:
            self.ndims = X_shape[1]
        else:         
            raise ValueError(f'sample_init() sample_X has incompatiple shape"{X_shape}" for the gaussian fit method.')
        
        self.sample_X = X.copy()
        self.sample_y = y.copy()


        self.gp.fit(self.sample_X, self.sample_y)

        sample_pred , sample_std = self.gp.predict(self.sample_X, return_std=True)

        return sample_pred

    def predict(self, grid_X):
                           
        X_len = len(grid_X.shape)

        if X_len == 1 and self.ndims == 1:
            pass      
        elif X_len ==2 and grid_X.shape[1] == self.ndims:       
            pass        
        else:
            raise ValueError(f'predict() grid_X has shape "{grid_X.shape}" dimensions. ndims = {self.ndims}')   

        grid_pred, grid_std = self.gp.predict(grid_X, return_std=True)

        return grid_pred, grid_std

    def log_marginal_likelihood(self, theta=None,eval_gradient=False, clone_kernel=True):

        return self.gp.log_marginal_likelihood( theta=theta, eval_gradient=eval_gradient,clone_kernel=clone_kernel  )
    
    def get_params(self):
        ic(self.gp.get_params(deep=True))

    def sample_add(self, X1, y1):

        if len(X1) != self.ndims: 
            raise ValueError(f'sample_add(). Error sample_X1 must have exactly "{self.ndims}"')      

        # add point to array
        X = np.concatenate((self.sample_X, [X1]),axis=0 )
        y = np.concatenate((self.sample_y, [y1]),axis=0)     

        # ic(X.shape,y.shape)
        # ic(X,y)
        
        # remember if sucessfull this will update self.sample_X, self.sample_y
        self.sample_init(X,y)

        return
    
    def gp_minimize(self):

        def surrogate_function(inputs):
            # Predict using the Gaussian Process surrogate model
            y_pred, stdev = self.gp.predict(np.atleast_2d(inputs), return_std=True) 
            return y_pred[0]  



        my_domain = [Real(0.0, 1.0, name=f'num_{self.ndims}') for i in range(self.ndims)]    
        
        res = skopt.gp_minimize(
            func=surrogate_function,
            dimensions=my_domain,  # The search space
            # base_estimator= self.gp, 
            acq_func=self.acquisition_function,
            kappa= self.kappa,
            n_calls = 4,  # high to be on the safe side
            
            x0=[list(x) for x in self.sample_X], # Sample X values
            y0= self.sample_y,                   # y target 
            verbose=False                         # debug output ??
        )
        
        print(f"gp_minimize() res.x = {res.x}")
        print(f"gp_minimize() res.fun = {res.fun}")

        suggested = np.array(res.x_iters[-1])
        print("Suggested next inputs to try:", suggested)

        del res  

        return

    def op_minimize(self,X, base_estimator  ):

        if X is None:
            X=np.array([])
         
        values = np.array([])
        my_domain = [Real(0.0, 1.0, name=f'num_{self.ndims}') for i in range(self.ndims)]    

            
        # if self.acquisition_function == "LCB":
        #     acq_func_kwargs = {"kappa": self.kappa}
        # elif self.acquisition_function == "EI" or   self.acquisition_function == "PI":
        #     acq_func_kwargs = {"xi": self.xi}
 
        acq_func_kwargs = {"xi": self.xi, "kappa": self.kappa}      
          
        opt = skopt.Optimizer(
            my_domain,
            base_estimator= self.gp  if base_estimator==None else base_estimator,
            acq_func=self.acquisition_function,
            n_initial_points=1,   
            random_state=0,
            acq_func_kwargs=acq_func_kwargs,
            acq_optimizer='sampling' 
        )
        
        opt.tell(self.sample_X.tolist()   , self.sample_y.tolist(), fit=True)
        possibles = opt.ask()

        # remember opt.models[0] is the base_optimizer in opt
        
        ic(opt.models[0])
        if len(X)>0:
            match self.acquisition_function:
                case "LCB":    
                    values = skopt.acquisition.gaussian_lcb(X, opt.models[0], kappa=self.kappa )    
                case "EI": 
                    values = skopt.acquisition.gaussian_ei(X, opt.models[0], y_opt=np.min(self.sample_y), xi=self.xi )
                case "PI":
                    values = skopt.acquisition.gaussian_pi(X, opt.models[0], y_opt=np.min(self.sample_y), xi=self.xi )    
                case _:
                    raise ValueError(f"op_minimize() unknown acquisition function {self.acquisition_function}") 


        
        # print(f"op_minimize(): value = {values}")         
        # print(f"opt.ask()={possibles}")

        del opt

        return values, possibles
  




In [ ]:


def calc(idx, add):
      
    sample_X, sample_y = load.load_function(idx,add)
    X_plot = None

    match idx:
          
        case 1:
            kernel = kernels.Matern(nu=2.5, length_scale=0.08 )
            g = GaussianProcessRegressor(kernel=kernel,n_restarts_optimizer=10)
            gp = mygp(g, kappa=1.0, acquisition_function="LCB", xi=0.01 )
            gp.sample_init(sample_X,sample_y)
            values, possibles = gp.op_minimize(X_plot, None)

        case 2:
            kernel = kernels.Matern(nu=2.5,length_scale=0.01)
            g = GaussianProcessRegressor(kernel=kernel,n_restarts_optimizer=10)
            gp= mygp(g, kappa=1, acquisition_function="EI",xi=0.1 )
            gp.sample_init(sample_X,sample_y)
            values, possibles=gp.op_minimize(X_plot,"GP")

        case 3: 
            sample_y*=-1.0
            kernel = kernels.Matern(nu=2.5,length_scale=0.01)
            g = GaussianProcessRegressor(kernel=kernel,n_restarts_optimizer=10)
            gp = mygp(g, kappa=0.5, acquisition_function="EI", xi=0.1 )
            gp.sample_init(sample_X,sample_y)
            values, possibles =gp.op_minimize(X_plot, None)

        case 4:
            kernel = kernels.Matern(nu=2.5,length_scale=0.01)
            g = GaussianProcessRegressor(kernel=kernel,n_restarts_optimizer=10)
            gp = mygp(g, kappa=1.0, acquisition_function="EI", xi=0.1)
            gp.sample_init(sample_X,sample_y)
            values, possibles = gp.op_minimize(X_plot, None)          
            
        case 5:
            kernel = kernels.Matern(nu=2.5,length_scale=0.01)
            g = GaussianProcessRegressor(kernel=kernel,n_restarts_optimizer=10)
            gp = mygp(g, kappa=1.0, acquisition_function="EI", xi=0.1)
            gp.sample_init(sample_X,sample_y)
            values, possibles = gp.op_minimize(X_plot, None)
            
        case 6:
            # sample_y *= -1.0
            kernel = kernels.Matern(nu=2.5,length_scale=0.01)
            g = GaussianProcessRegressor(kernel=kernel,n_restarts_optimizer=10)
            gp= mygp(g, kappa=1.0, acquisition_function="LCB", xi=0.1)
            gp.sample_init(sample_X,sample_y)
            values, possibles = gp.op_minimize(X_plot, None)
        case 7:
            sample_y *= -1.0    
            kernel = kernels.Matern(nu=2.5,length_scale=0.01)
            g = GaussianProcessRegressor(kernel=kernel,n_restarts_optimizer=10)
            gp = mygp(g, kappa=2.0, acquisition_function="LCB", xi=0.1)
            gp.sample_init(sample_X,sample_y)
            values, possibles = gp.op_minimize(None, "GP")
        case 8: 
            sample_y *= -1.0
            kernel = kernels.Matern(nu=2.5,length_scale=0.01)
            g = GaussianProcessRegressor(kernel=kernel,n_restarts_optimizer=10)
            gp = mygp(g, kappa=2.0, acquisition_function="EI", xi=0.1)
            gp.sample_init(sample_X,sample_y)
            values, possibles = gp.op_minimize(None, None)

    ic(possibles)    
    if ic.enabled:
        gp.prn()
        
    return possibles
    

In [14]:



ic.disable()

possibles_list=[]

for idx in range(1,9):
    print(f"idx={idx}")
    possibles=calc(idx,8)
    possibles_list.append(possibles)



load.print_for_capstone (possibles_list)


idx=1
idx=2


/home/hmb/anaconda3/envs/ts/lib/python3.10/site-packages/sklearn/gaussian_process/_gpr.py:616: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)


idx=3
idx=4
idx=5
idx=6


/home/hmb/anaconda3/envs/ts/lib/python3.10/site-packages/sklearn/gaussian_process/_gpr.py:616: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)


idx=7


/home/hmb/anaconda3/envs/ts/lib/python3.10/site-packages/sklearn/gaussian_process/_gpr.py:616: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)


idx=8
0.000546-0.721183-
0.354506-0.970611-
0.989766-0.929733-0.995595-
0.994375-0.955254-0.886629-0.696208-
0.827452-0.585675-0.151086-0.400371-
0.147108-0.942920-0.070672-0.083941-0.950822-
0.289776-0.003340-0.340427-0.253258-0.446117-0.737255-
0.004768-0.075287-0.324064-0.084006-0.809979-0.791310-0.229669-0.740011-


0.000367-0.005225- <BR>
0.122140-0.031826- <BR>
0.983496-0.001545-0.986330- <BR>
0.987889-0.837106-0.856178-0.695794- <BR>
0.496921-0.779008-0.269521-0.129492- <BR>
0.001819-0.959575-0.030757-0.019916-0.950110- <BR>
0.014083-0.783947-0.254345-0.278122-0.454798-0.171580- <BR>
0.061388-0.174398-0.144627-0.311616-0.580443-0.120990-0.228950-0.195130- <BR>

In [19]:
ic.enable()
p = calc(1,9)

a,b = load.load_function(1,9)

ic(np.max(b))

/home/hmb/anaconda3/envs/ts/lib/python3.10/site-packages/sklearn/gaussian_process/_gpr.py:616: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)
ic| opt.models[0]: GaussianProcessRegressor(kernel=Matern(length_scale=0.01, nu=2.5),
                                            n_restarts_optimizer=10)
ic| possibles: [0.0005459648969956544, 0.721182977494583]
ic| np.max(b): 2.6752879910742468e-09


ndims = 2. sample shape = (16, 2)
kappa=1.0
xi=0.01
acquisition_function=LCB
[[3.19403889e-01 7.62959374e-01]
 [5.74329215e-01 8.79898105e-01]
 [7.31023631e-01 7.32999876e-01]
 [8.40353417e-01 2.64731614e-01]
 [6.50114060e-01 6.81526352e-01]
 [4.10437137e-01 1.47554299e-01]
 [3.12691157e-01 7.87227779e-02]
 [6.83418169e-01 8.61057464e-01]
 [8.25072518e-02 4.03487506e-01]
 [8.83889829e-01 5.82253974e-01]
 [6.51000000e-01 6.80000000e-01]
 [6.51000000e-01 6.70000000e-01]
 [5.00000000e-01 5.00000000e-01]
 [1.46900000e-03 9.97603000e-01]
 [9.96882000e-01 1.17740000e-02]
 [3.67000000e-04 5.22500000e-03]]
[ 1.32267704e-079  1.03307824e-046  7.71087511e-016  3.34177101e-124
 -3.60606264e-003 -2.15924904e-054 -2.08909327e-091  2.53500115e-040
  3.60677119e-081  6.22985647e-048 -3.76257505e-003 -3.10770634e-003
  2.67528799e-009  0.00000000e+000  0.00000000e+000  6.20515754e-245]


2.6752879910742468e-09